# Pipeline Principal — Inferencia por Cluster

Flujo completo:
1. Configurar parámetros (cluster, número de cajeros, horizonte)
2. Cargar datos históricos
3. Clasificar cajeros y seleccionar los del cluster elegido
4. Cargar el modelo entrenado
5. Ejecutar inferencia in-sample + futura
6. Ver gráfica real vs predicción por cajero
7. Resumen de métricas

**Clusters disponibles:** `normal_estable`, `normal_con_picos`, `event_driven`, `grande_y_estable`

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

DATA_PATH   = PROJECT_ROOT / "insumos" / "df_general.parquet"
MODELO_PATH = PROJECT_ROOT / "models"  / "neuralprophet.np"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Datos:       ", DATA_PATH,   "— existe:", DATA_PATH.exists())
print("Modelo:      ", MODELO_PATH, "— existe:", MODELO_PATH.exists())

## Configuración — modifica aquí

In [ ]:
# ============================================================
# PARÁMETROS DEL USUARIO
# ============================================================

# Cluster a analizar.
# Opciones: "normal_estable" | "normal_con_picos" | "event_driven" | "grande_y_estable"
CLUSTER = "normal_estable"

# Número de cajeros a seleccionar de ese cluster
N_CAJEROS = 3

# Modo de selección de cajeros
# Opciones: "aleatorio" | "mayor_mediana" | "mayor_ratio"
MODO_SELECCION = "aleatorio"

# Días a predecir hacia el futuro (se muestra como línea punteada en la gráfica)
HORIZONTE = 30

print(f"Cluster:       {CLUSTER}")
print(f"N cajeros:     {N_CAJEROS}")
print(f"Modo:          {MODO_SELECCION}")
print(f"Horizonte:     {HORIZONTE} días")

## Imports

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from src.segmentation import (
    construir_resumen_estructural,
    aplicar_reglas_precluster,
    seleccionar_cajeros_por_cluster
)
from src.inference import inferencia_por_lista
from src.events_calendar import CALENDARIOS_EXACTOS

# Fechas únicas de pago para las líneas verticales de la gráfica
FECHAS_PAGO = sorted({
    pd.Timestamp(fecha_str)
    for dias in CALENDARIOS_EXACTOS.values()
    for fecha_str, _ in dias
})

print("Imports OK")
print(f"Fechas de pago en el calendario: {len(FECHAS_PAGO)}")

## 1. Cargar datos históricos

In [ ]:
df = pd.read_parquet(DATA_PATH)

print(f"Filas: {len(df):,}")
print(f"Cajeros únicos: {df['cajero'].nunique():,}")
print(f"Período: {df['fecha'].min()} → {df['fecha'].max()}")

## 2. Clasificar cajeros y seleccionar cluster

In [ ]:
resumen = construir_resumen_estructural(df)
df_clasificados = aplicar_reglas_precluster(resumen)

print("Distribución de clusters:")
print(df_clasificados["etiqueta"].value_counts().to_string())
print()

df_cluster_sel = seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster=CLUSTER,
    n=N_CAJEROS,
    modo=MODO_SELECCION
)

lista_cajeros = df_cluster_sel["cajero"].tolist()

print(f"Cajeros seleccionados del cluster '{CLUSTER}':")
print(df_cluster_sel[["cajero", "dias_obs", "mediana", "ratio_p95_mediana", "etiqueta"]].to_string(index=False))

## 3. Cargar modelo entrenado

In [ ]:
if not MODELO_PATH.exists():
    raise FileNotFoundError(
        f"No se encontró el modelo en:\n  {MODELO_PATH}\n"
        "Ejecuta primero el notebook de entrenamiento: Scripts/prueba_neuro_prophet.ipynb"
    )

print(f"Modelo encontrado: {MODELO_PATH}")

## 4. Ejecutar inferencia

In [ ]:
print(f"Iniciando inferencia para {len(lista_cajeros)} cajero(s) del cluster '{CLUSTER}'...")
print(f"Horizonte futuro: {HORIZONTE} días\n")

resultados = inferencia_por_lista(
    lista_cajeros=lista_cajeros,
    df_historico=df,
    ruta_modelo=MODELO_PATH,
    horizonte=HORIZONTE
)

print(f"\nInferencia completada: {len(resultados)} cajero(s) procesados.")

## 5. Resultados por cajero — gráfica y métricas

In [ ]:
if not resultados:
    print("No hay resultados. Revisa que los cajeros tengan al menos 150 días de historial.")
else:
    for cajero_id, datos in resultados.items():
        merged   = datos["merged_historico"]
        pred_fut = datos["predicciones_futuras"]
        m        = datos["metricas"]

        # ── Imprimir métricas ─────────────────────────────────────────────
        print(f"{'='*70}")
        print(f"  Cajero: {cajero_id}  |  Cluster: {CLUSTER}")
        print(f"{'='*70}")
        print(f"  Días históricos con predicción: {len(merged)}")
        print(f"  Predicciones negativas corregidas a 0: {m['n_negativos_corregidos']}")
        print()
        print(f"  R²   (coef. de determinación):  {m['r2']:.4f}")
        print(f"  MAE  (error absoluto medio):    ${m['mae']:>12,.0f}")
        print(f"  RMSE (raíz error cuadrático):   ${m['rmse']:>12,.0f}")
        print()

        # ── Construir gráfica ─────────────────────────────────────────────
        fig, ax = plt.subplots(figsize=(20, 6))

        # Serie real
        ax.plot(
            merged["ds"], merged["y"],
            label="Real", color="black", linewidth=1.3, alpha=0.85
        )

        # Predicción in-sample
        ax.plot(
            merged["ds"], merged["yhat1"],
            label=f"Predicción in-sample (R²={m['r2']:.4f})",
            color="red", linewidth=1.3, alpha=0.85
        )

        # Predicciones futuras (línea punteada azul)
        if len(pred_fut) > 0:
            yhat_col = "yhat1" if "yhat1" in pred_fut.columns else pred_fut.columns[-1]
            # Conectar con el último punto histórico para que no quede flotando
            ult_ds  = merged["ds"].iloc[-1]
            ult_y   = merged["yhat1"].iloc[-1]
            ds_ext  = pd.concat([pd.Series([ult_ds]),  pred_fut["ds"]]).reset_index(drop=True)
            y_ext   = pd.concat([pd.Series([ult_y]),   pred_fut[yhat_col]]).reset_index(drop=True)
            ax.plot(
                ds_ext, y_ext,
                label=f"Futuro ({HORIZONTE} días)",
                color="blue", linewidth=1.8, linestyle="--", alpha=0.9
            )

        # Líneas verticales en fechas de pago
        ds_min = merged["ds"].min()
        ds_max = merged["ds"].max()
        for fecha in FECHAS_PAGO:
            if ds_min <= fecha <= ds_max:
                ax.axvline(fecha, color="blue", alpha=0.18, linewidth=0.7)

        # Formato
        ax.set_title(
            f"NeuralProphet | Cajero {cajero_id} | Cluster: {CLUSTER}",
            fontsize=14
        )
        ax.set_xlabel("Fecha")
        ax.set_ylabel("Retiro ($)")
        ax.yaxis.set_major_formatter(
            plt.FuncFormatter(lambda x, _: f"${x:,.0f}")
        )
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
        plt.xticks(rotation=45)
        ax.legend(loc="upper left")
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        print()

## 6. Resumen de métricas

In [ ]:
if not resultados:
    print("Sin resultados para mostrar.")
else:
    filas = []
    for cajero_id, datos in resultados.items():
        m = datos["metricas"]
        filas.append({
            "Cajero":             cajero_id,
            "Cluster":            CLUSTER,
            "Días históricos":    datos["n_dias_historicos"],
            "Media retiro ($)":   f"{datos['media_historica']:,.0f}",
            "R²":                 m["r2"],
            "MAE ($)":            f"{m['mae']:,.0f}",
            "RMSE ($)":           f"{m['rmse']:,.0f}",
            "Neg. corregidos":    m["n_negativos_corregidos"],
        })

    df_resumen = pd.DataFrame(filas)

    print(f"RESUMEN DE MÉTRICAS — Cluster '{CLUSTER}'")
    print("=" * 80)
    print(df_resumen.to_string(index=False))
    print()

    r2_prom   = df_resumen["R²"].mean()
    mejor_r2  = df_resumen.loc[df_resumen["R²"].idxmax(), "Cajero"]
    peor_r2   = df_resumen.loc[df_resumen["R²"].idxmin(), "Cajero"]

    print(f"R² promedio del cluster: {r2_prom:.4f}")
    print(f"Mejor ajuste:  cajero {mejor_r2}")
    print(f"Peor ajuste:   cajero {peor_r2}")